### Parsing multiple CSV files into one single CSV file (for both)
#### Also, formating data types (from str to int or float) (for both)
#### Also, date formating from 12/22/15 to 2015-12-22 (for Non CZ files only)


## CZ Files

The following notebook takes all files contained in `../config/paths.yaml` file (under `cz_files`) and unifies them into one. 
The files are combined (they have matching columns), but also there is data transformation going on. 
From the same entry (say one hospital admision) there are multiple rows in the original data set. For example, when a patient has multiple diagnosis the table looks like:


| Admision Date      | Disch Date | Diagnosis     | 
| ------------------ | ---------- | --------------|
| x1    | y1         | diagnostic 1 for patient 1 |
| NaN   | NaN        |  diagnostic 2 for patient 1|
| NaN   | NaN        |  diagnostic 3 for patient 1|
| x2    | y2         |  diagnostic 1 for patient 2|
| NaN   | NaN        |  diagnostic 2 for patient 2|
| ...   | ...        |  ...                       |

Table is transform such that all diagnostic are in the same row:
| Admision Date      | Disch Date | Diagnosis     | 
| ------------------ | ---------- | --------------|
| x1    | y1         | [diagnostic 1 for patient 1, Diagnostic 1 for patient 1, Diagnostic 3 for patient1 ]|
| x1    | y1         | [diagnostic 1 for patient 2, Diagnostic 1 for patient 2, ... ]|
| ...   | ...        |  ...                       |

Same process is applied to all columns that have multiple rows for same patients (are unified into one cell as a list)

### Imports

In [1]:
import os
import yaml
import pandas as pd
import numpy as np
import re

### Retrieving file paths and config

In [2]:
with open('../config/paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

filepaths = [os.path.join(config['data_path'], file) for file in config['cz_files']]
assert all([os.path.isfile(filepath) for filepath in filepaths])


filepaths[0]

'/Users/marianomaisonnave/Documents/CBU Postdoc/Grant Data/Merged/2015/ALC Machine 2015Q1 - coded HCN.csv'

### Reading all CSVs

In [ ]:
all_dfs = []
for filepath in (filepaths):
    df = pd.read_csv(filepath)
    all_dfs.append(df)

### Merging data into one DataFrame (`df`)

In [67]:
column_count = all_dfs[0].shape[1]
assert all([df.shape[1]==column_count for df in all_dfs])
print(f'Column count={column_count}')


for column_ix in range(column_count):
    print(f'{column_ix:2}: {set([df.columns[column_ix] for df in all_dfs])}')
    assert len(set([df.columns[column_ix] for df in all_dfs]))==1

df = pd.concat(all_dfs)

print(f'Shape of merged data {df.shape[0]:,}x{df.shape[1]}')

Column count=27
 0: {'Institution Number:'}
 1: {'Coded HCN'}
 2: {'Admit Date:'}
 3: {'Disch Date:'}
 4: {'Readmission Code:'}
 5: {'Patient Age:'}
 6: {'Gender:'}
 7: {'MRDx'}
 8: {'Postal Code:'}
 9: {'Diagnosis:'}
10: {'Diagnosis Long Text'}
11: {'Diagnosis Type'}
12: {'Intervention Code'}
13: {'Px Long Text'}
14: {'Admit Category:'}
15: {'Entry Code:'}
16: {'Transfusion Given'}
17: {'Main Pt Service:'}
18: {'CMG'}
19: {'Comorbidity Level'}
20: {'Case Weight'}
21: {'ALCDays'}
22: {'Acute Days'}
23: {'Institution To'}
24: {'Institution From'}
25: {'Institution Type'}
26: {'Discharge Nurse Unit'}
Shape of merged data 1,430,466x27


### Combininig rows that belong to the same data entry. 

In [174]:


data = {}
for column in df.columns:
    data[column]=[]
row_ix=0
while row_ix<df.shape[0]:
    end = row_ix + 1
    while end<df.shape[0] and np.isnan(df.iloc[end, 0]) :
        end+=1

    auxdf = df.iloc[row_ix:end, :].copy()
    
    for column in auxdf:
        content = [elem for ix,elem in enumerate(auxdf[column]) if not auxdf[column].isna().iloc[ix]]
        content = content[0] if len(content)==1 else content
        data[column].append(content)

    row_ix=end   
    
newdf=pd.DataFrame(data)


### Trasnfroming data types (from str to int or float)

In [ ]:
newdf['ALCDays'] = [re.sub(',','',alcdays) if isinstance(alcdays,str) else alcdays for alcdays in newdf['ALCDays']]
newdf['Acute Days'] = [re.sub(',','',alcdays) if isinstance(alcdays,str) else alcdays for alcdays in newdf['Acute Days']]
newdf = newdf.astype({'Institution Number:': 'int',  
                      'Patient Age:': 'int', 
                      'ALCDays':'int',
                      'Acute Days': 'int',
                       })

# Making Case Weight a float
new_case_weight=[]
for elem in newdf['Case Weight']:
    if elem==[]:
        newelem=np.nan
    elif isinstance(elem, str):
        assert '.' in elem
        newelem =  float(elem.replace(',',''))
    else:
        assert isinstance(elem,float)
        newelem = elem
    new_case_weight.append(newelem)
newdf['Case Weight'] = np.array(new_case_weight)


### Checking contents of columns and saving to disk

In [ ]:
for column,value in zip(newdf.columns, newdf.iloc[0,:]):
    # count=len(set(df[column]))
    print(f'{column:20} ({str(type(value)):23}):   {value}')

In [ ]:
newdf.to_csv(config['unified_merged_file_cz'], index=None)

### Formatting dates not required for CZ files

In [23]:
pd.read_csv(config['unified_merged_file_cz'])

/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_18289/2350669175.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(config['unified_merged_file_cz'])


,Institution Number:,Coded HCN,Admit Date:,Disch Date:,Readmission Code:,Patient Age:,Gender:,MRDx,Postal Code:,Diagnosis:,...,Main Pt Service:,CMG,Comorbidity Level,Case Weight,ALCDays,Acute Days,Institution To,Institution From,Institution Type,Discharge Nurse Unit
0,65,1159480.0,2014/12/24,2015/01/01,5 New patient to the acute care unit,67,Male,K746 Other and unspecified cirrhosis of liver,B3A2K1,"['K746', 'K431', 'R18', 'K760', 'K650', 'I518'...",...,Family Practice,285,1 Level 1 Comorbidity,1.3894,0,8,QE11 Health Sciences Centre,"Home Care NS - CIHI Disch Dispn 04, Dialysis, VON","ACUTE, ACUTE PSYCHIATRIC (0)",D42
1,65,2232961.0,2014/12/30,2015/01/01,5 New patient to the acute care unit,38,Female,K8010 Calculus of gallbladder with other chol...,B3E1N7,"['K8010', 'Z720']",...,General Surgery,278,0 No Significant Comorbidity,0.6825,0,2,**,**,**,D32
2,65,9938495.0,2014/12/29,2015/01/01,1 Planned readmit from prev acute care (no tim...,62,Male,T8404 Mechanical complication of knee prosthesis,B2Y2G6,"['T8404', 'M2546', 'Y831']",...,Orthopaedic Surgery,319,0 No Significant Comorbidity,1.8371,0,3,"Home Care NS - CIHI Disch Dispn 04, Dialysis, VON",**,"OTHER (9002,9003,9088,9089)",D32
3,65,12350498.0,2014/12/31,2015/01/01,9 Other (none of the above),37,Female,"S82810 Trimalleolar fracture of ankle, closed",B2W2R6,"['S82810', 'X590', 'U984', 'E109']",...,Orthopaedic Surgery,729,0 No Significant Comorbidity,1.0085,0,1,**,**,**,D32
4,65,8000546788.0,2014/12/29,2015/01/01,5 New patient to the acute care unit,49,Male,"M179 Gonarthrosis, unspecified",B0S1C0,"['M179', 'I100', 'Z720']",...,Orthopaedic Surgery,321,0 No Significant Comorbidity,1.4334,0,3,"Home Care NS - CIHI Disch Dispn 04, Dialysis, VON",**,"OTHER (9002,9003,9088,9089)",D32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256001,77,7759382.0,2022/07/28,2022/12/28,1 Planned readmit from prev acute care (no tim...,37,Female,Z751 Person awaiting admission to adequate fa...,B0J1N0,Z751,...,Alternate Level of Care,809.0,0 No Significant Comorbidity,13.9496,153,0,**,**,**,NB2
256002,77,13519159.0,2022/12/21,2022/12/28,1 Planned readmit from prev acute care (no tim...,41,Male,"F209 Schizophrenia, unspecified",B3A1A1,F209,...,Psychiatry,707.0,0 No Significant Comorbidity,1.9760,0,7,COMMUNITY MENTAL HEALTH SERVICES-COMM MH ADDICT,COMMUNITY MENTAL HEALTH SERVICES-COMM MH ADDICT,**,NMU
256003,77,4956361.0,2022/12/22,2022/12/29,1 Planned readmit from prev acute care (no tim...,43,Female,F103 Mental and behavioural disorders due to ...,B3V1L1,"['F103', 'E834']",...,Family Practice,708.0,0 No Significant Comorbidity,0.5822,0,7,**,**,**,NAD
256004,77,2911012.0,2022/12/21,2022/12/30,1 Planned readmit from prev acute care (no tim...,35,Male,F143 Mental and behavioural disorders due to ...,B3S0G8,"['F143', 'F103', 'F900', 'F419', 'K259']",...,Family Practice,708.0,1 Level 1 Comorbidity,0.8028,0,9,**,**,**,NAD


## Non CZ Files

The following notebook takes all files contained in `../config/paths.yaml` file (under `noncz_files`) and unifies them into one. 
The files are combined (they have matching columns), but also there is data transformation going on. 
From the same entry (say one hospital admision) there are multiple rows in the original data set. For example, when a patient has multiple diagnosis the table looks like:


| Admision Date      | Disch Date | Diagnosis     | 
| ------------------ | ---------- | --------------|
| x1    | y1         | diagnostic 1 for patient 1 |
| NaN   | NaN        |  diagnostic 2 for patient 1|
| NaN   | NaN        |  diagnostic 3 for patient 1|
| x2    | y2         |  diagnostic 1 for patient 2|
| NaN   | NaN        |  diagnostic 2 for patient 2|
| ...   | ...        |  ...                       |

Table is transform such that all diagnostic are in the same row:
| Admision Date      | Disch Date | Diagnosis     | 
| ------------------ | ---------- | --------------|
| x1    | y1         | [diagnostic 1 for patient 1, Diagnostic 1 for patient 1, Diagnostic 3 for patient1 ]|
| x1    | y1         | [diagnostic 1 for patient 2, Diagnostic 1 for patient 2, ... ]|
| ...   | ...        |  ...                       |

Same process is applied to all columns that have multiple rows for same patients (are unified into one cell as a list)


### Imports

In [4]:
import pandas as pd
import numpy as np
import yaml
import os
import re 

### Retrieving file paths and config

In [5]:
with open('../config/paths.yaml', 'r') as file:
    config = yaml.safe_load(file)
print(config['data_path'])
print(config['noncz_files'][0])
filepaths = [os.path.join(config['data_path'], file) for file in config['noncz_files']]
assert all([os.path.isfile(filepath) for filepath in filepaths])

/Users/marianomaisonnave/Documents/CBU Postdoc/Grant Data/Merged/
2015/2015 Non CZ - coded HCN/noncz 2015Q1.csv


### Reading all CSVs

In [8]:
all_dfs = []
for filepath in (filepaths):
    df = pd.read_csv(filepath)
    all_dfs.append(df)


/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_16786/311908344.py:3: DtypeWarning: Columns (18,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_16786/311908344.py:3: DtypeWarning: Columns (4,18,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_16786/311908344.py:3: DtypeWarning: Columns (18,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_16786/311908344.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_16786/311908344.py:3: DtypeWarning: Columns (4,18,20,22) have mixed types. Sp

### Merging data into one DataFrame (`df`)

In [9]:
column_count = all_dfs[0].shape[1]
assert all([df.shape[1]==column_count for df in all_dfs])
print(f'Column count={column_count}')


for column_ix in range(column_count):
    print(f'{column_ix:2}: {set([df.columns[column_ix] for df in all_dfs])}')
    assert len(set([df.columns[column_ix] for df in all_dfs]))==1, set([df.columns[column_ix] for df in all_dfs])

df = pd.concat(all_dfs)

print(f'Shape of merged data {df.shape[0]:,}x{df.shape[1]}')

Column count=27
 0: {'Institution Number:'}
 1: {'Coded HCN'}
 2: {'Admit Date:'}
 3: {'Disch Date:'}
 4: {'Readmission Code:'}
 5: {'Patient Age:'}
 6: {'Gender:'}
 7: {'MRDx'}
 8: {'Postal Code:'}
 9: {'Diagnosis:'}
10: {'Diagnosis Long Text'}
11: {'Diagnosis Type'}
12: {'Intervention Code'}
13: {'Px Long Text'}
14: {'Admit Category:'}
15: {'Entry Code:'}
16: {'Transfusion Given'}
17: {'Main Pt Service:'}
18: {'CMG'}
19: {'Comorbidity Level'}
20: {'Case Weight'}
21: {'ALCDays'}
22: {'Acute Days'}
23: {'Institution To'}
24: {'Institution From'}
25: {'Inst Type 2018'}
26: {'Nursing Unit:'}
Shape of merged data 1,429,894x27


### Combininig rows that belong to the same data entry. 

In [10]:
data = {}
for column in df.columns:
    data[column]=[]
row_ix=0
while row_ix<df.shape[0]:
    end = row_ix + 1
    while end<df.shape[0] and np.isnan(df.iloc[end, 0]) :
        end+=1

    auxdf = df.iloc[row_ix:end, :].copy()
    
    for column in auxdf:
        content = [elem for ix,elem in enumerate(auxdf[column]) if not auxdf[column].isna().iloc[ix]]
        content = content[0] if len(content)==1 else content
        data[column].append(content)

    row_ix=end   
    
newdf=pd.DataFrame(data)


In [19]:

newdf['ALCDays'] = [re.sub(',','',alcdays) if isinstance(alcdays,str) else alcdays for alcdays in newdf['ALCDays']]
newdf['Acute Days'] = [re.sub(',','',alcdays) if isinstance(alcdays,str) else alcdays for alcdays in newdf['Acute Days']]
newdf = newdf.astype({'Institution Number:': 'int',  
                      'Patient Age:': 'int', 
                      'ALCDays':'int',
                      'Acute Days': 'int',
                       })

In [20]:
for column,value in zip(newdf.columns, newdf.iloc[0,:]):
    # count=len(set(df[column]))
    print(f'{column:20} ({str(type(value)):23}):   {value}')

Institution Number:  (<class 'numpy.int64'>  ):   43
Coded HCN            (<class 'list'>         ):   [4988519.0, 170805.0, 170805.0]
Admit Date:          (<class 'str'>          ):   12/22/14
Disch Date:          (<class 'str'>          ):   1/1/15
Readmission Code:    (<class 'str'>          ):   9
Patient Age:         (<class 'numpy.int64'>  ):   80
Gender:              (<class 'str'>          ):   Female
MRDx                 (<class 'str'>          ):   I500  Congestive heart failure
Postal Code:         (<class 'str'>          ):   B2G1Y1
Diagnosis:           (<class 'list'>         ):   ['I500', 'E1152', 'J449']
Diagnosis Long Text  (<class 'list'>         ):   ['Congestive heart failure', 'Type 2 diabetes mellitus with certain circulatory complications', 'Chronic obstructive pulmonary disease, unspecified']
Diagnosis Type       (<class 'list'>         ):   ['M', '3', '3']
Intervention Code    (<class 'str'>          ):   **
Px Long Text         (<class 'str'>          ):   **
A

In [22]:
newdf.to_csv(config['unified_merged_file_noncz'], index=None)

In [21]:
config['unified_merged_file_noncz']

'/Users/marianomaisonnave/Documents/CBU Postdoc/Grant Data/Merged/2015_2022/full_noncz_database.csv'

### Reading from disk, formating dates and saving again

In [21]:
df = pd.read_csv(config['unified_merged_file_noncz'])
df.head()
admit_dates=[]
for date in df['Admit Date:']:
    assert len(date.split('/'))==3 or date=='**', date
    if len(date.split('/'))==3:
        month, day, year = date.split('/')    
        if 1<=int(year) and int(year)<=23:
            newdate=f'20{year}-{month}-{day}'
        else:
            newdate=f'19{year}-{month}-{day}'
    else:
        assert date=='**'
        newdate=date
    admit_dates.append(newdate)
df['Admit Date:']=admit_dates

disch_dates=[]
for date in df['Disch Date:']:
    assert len(date.split('/'))==3 or date=='**', date
    if len(date.split('/'))==3:
        month, day, year = date.split('/')    
        assert 15<=int(year) and int(year)<=22
        newdate=f'20{year}-{month}-{day}'

    else:
        assert date=='**'
        newdate=date
    disch_dates.append(newdate)
df['Disch Date:']=disch_dates
df.to_csv(config['unified_merged_file_noncz'], index=None)
df.head()

/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_18289/2572917841.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config['unified_merged_file_noncz'])


,Institution Number:,Coded HCN,Admit Date:,Disch Date:,Readmission Code:,Patient Age:,Gender:,MRDx,Postal Code:,Diagnosis:,...,Main Pt Service:,CMG,Comorbidity Level,Case Weight,ALCDays,Acute Days,Institution To,Institution From,Inst Type 2018,Nursing Unit:
0,43,"[4988519.0, 170805.0, 170805.0]",2014-12-22,2015-1-1,9,80,Female,I500 Congestive heart failure,B2G1Y1,"['I500', 'E1152', 'J449']",...,Cardiology,196.0,0 No Significant Comorbidity,0.8487,0,10,9089,**,**,**
1,43,"[5900259.0, 170805.0, 170805.0]",2014-12-30,2015-1-1,9,60,Male,"K2214 Ulcer of oesophagus, chronic or unspeci...",B0E2G0,"['K2214', 'K295', 'B980']",...,General Surgery,254.0,0 No Significant Comorbidity,0.6702,0,2,**,68,**,**
2,43,"[8972535.0, 170805.0]",2014-12-24,2015-1-1,9,80,Male,J441 Chronic obstructive pulmonary disease wi...,B2G2M5,"['J441', 'J189']",...,General Medicine,139.0,0 No Significant Comorbidity,0.7441,0,8,**,**,**,**
3,43,"[1680780.0, 170805.0, 170805.0]",2014-12-27,2015-1-1,9,67,Female,"I639 Cerebral infarction, unspecified",B0E1A0,"['I639', 'E1152', 'D649']",...,General Medicine,26.0,0 No Significant Comorbidity,0.9942,0,5,**,**,**,**
4,43,"[2071649.0, 170805.0, 170805.0, 170805.0, 1708...",2014-12-29,2015-1-1,5,37,Female,O34201 Maternal care for uterine scar due to ...,B2G1S5,"['O34201', 'O14901', 'O24801', 'Z37000', 'Z302...",...,OBS Delivered,560.0,1 Level 1 Comorbidity,0.8487,0,3,**,**,**,**
